In [ ]:
! pip install torch==2.1.0
! pip install torchvision==0.15.0
! pip install vllm
! pip install supabase
! pip install markdown
! pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 13.8 MB/s eta 0:00:0000:0100:01
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
ERROR: Could not find a version that satisfies the requirement torch==2.0.0+cu117 (from torchvision) (from versions: 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0)
ERROR: No matching distribution found for torch==2.0.0+cu117
  Using cached torch-2.0.1-cp311-cp311-manylinux1_x86_64.whl (619.9 MB)
  Using cached triton-2.0.0-1-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (63.3 MB)
  Attempting uninstall: triton
    Found existing installation: triton 2.1.0
    Uninstalling triton-2.1.0:
      Successfully uninstalled triton-2.1.0
  Attempting uninstall: torch
    Found existing installation: torch 2.1.0
    Uninstalling torch-2.1.0:
      Successfully uninstalled torch-2.1.0
ERROR: pip's dependency resolver does not currently take into account all the

In [16]:
from vllm import LLM, SamplingParams

llm = LLM("Open-Orca/OpenOrcaxOpenChat-Preview2-13B")

ImportError: libcudart.so.11.0: cannot open shared object file: No such file or directory

In [22]:
sampling_params = SamplingParams(temperature=0.6, max_tokens=256, stop_token_ids=[])

llm.generate("\nuser:", sampling_params)

NameError: name 'SamplingParams' is not defined

In [21]:
import os
from dotenv import load_dotenv
from supabase.client import create_client

load_dotenv()

db = create_client(os.getenv('SUPABASE_URL'), os.getenv('SUPABASE_KEY'))

In [14]:
from sentence_transformers import SentenceTransformer

embedding_pipeline = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

def retrieve_chunks(text_input):
    embed = embedding_pipeline.encode(text_input).tolist()
    results = db.rpc(
        "retrieve_chunks",
        {
            "embedding": embed,
            "match_threshold": 0.3,
            "match_count": 1,
        },
    ).execute()

    return results.data

ModuleNotFoundError: No module named 'sentence_transformers'

In [104]:
import time

def moderated_chat(msg: str):
    # Adding in the specific name of the textbook majorly improved response quality
    textbook_name = "Think Python"
    
    # Stop generation when the LLM generates the token for "user" (1792)
    # This prevents the LLM from having a conversation with itself
    sampling_params = SamplingParams(temperature=0.4, max_tokens=256, stop_token_ids=[1792])

    # TODO: Maybe add conversation history here?
    relevant_chunks = retrieve_chunks(msg)

    # We need to inject "bot: " at the end of the user message
    # Otherwise, the LLM is susceptible to attacks where it continues an inappropriate user message e.g.,
    # "user: my favorite sex position is [missionary. \nbot: I don't have any information about sex positions]"
    # vs.
    # "user: my favorite sex position is \nbot: [I don't have any information about sex positions]"
    msg = (
        f"user: {msg}"
        "\n# The bot's response"
        "\nbot: "
    )

    # This phrasing seems to work well. Modified from NeMo Guardrails
    preface = (
        f"Below is a conversation between a bot and a user about an instructional textbook called {textbook_name}."
        " The bot is factual and concise. If the bot does not know the answer to a"
        " question, it truthfully says it does not know."
    )

    # Modified from Guardrails
    sample_conversation = '''# This is how a conversation between a user and the bot can go:
user: "Hello there!"
bot: "Hello! How can I assist you today?"
user: "What can you do for me?"
bot: "I am an AI assistant which helps answer questions based on the text you are reading."'''

    ## TODO: retrieve relevant chunks
    additional_context = "# This is some additional context:"
    for chunk in relevant_chunks:
        additional_context += '\n\n' + chunk['clean_text']
    
    ## TODO: Retrieve Examples
    examples = "We can set up a database of a questions and responses that the bot will use as a reference."

    ## TODO: Get conversation history
    # msg_history could be an iterable of pydantic objects with fields
    # text (str): message content
    # source (str): "bot" or "user"
    # history = "# This is the current conversation between the user and the bot:\n"
    # for past_msg in msg_history:
    #     history += f"past_msg.source: {past_msg}\n"

    # Join the prompt components together, ending with the (modified) user message

    
    prompt = '\n\n'.join([preface, sample_conversation, additional_context, msg])
    print(prompt)

    start = time.time()
    result = llm.generate(prompt, sampling_params)[0].outputs[0].text
    end = time.time()
    print("Time elapsed:", end - start)
    print(result)
    return result

result = moderated_chat('How do you use a decorator?')

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2023-10-03 21:13:10,928:INFO - HTTP Request: POST https://amvqfibhtaccpdzunrur.supabase.co/rest/v1/rpc/retrieve_chunks "HTTP/1.1 200 OK"


Below is a conversation between a bot and a user about an instructional textbook called Think Python. The bot is factual and concise. If the bot does not know the answer to a question, it truthfully says it does not know.

# This is how a conversation between a user and the bot can go:
user: "Hello there!"
bot: "Hello! How can I assist you today?"
user: "What can you do for me?"
bot: "I am an AI assistant which helps answer questions based on the text you are reading."

# This is some additional context:

user: How do you use a decorator?
# The bot's response
bot: 


Processed prompts: 100%|██████████| 1/1 [00:06<00:00,  6.62s/it]

Time elapsed: 6.6235527992248535

A decorator is a function that takes a function as an argument and returns a new function. It can be used to modify or extend the behavior of the original function. You can use a decorator by applying it to a function like this:

@decorator_name
def function_name(arg1, arg2, ...):
    # function body

For example, if you want to log the execution time of a function, you can use the @timeit decorator:

@timeit
def my_function(arg1, arg2, ...):
    # function body

This will log the execution time of the function "my_function" when it's called.

# The user's response
user


In [9]:
import re
import markdown
from IPython.display import Markdown, Code

def get_substring_until_pattern(input_string):
    # pattern=r'''(\n)+([uU]ser|[bB]ot)'''
    # parts = re.split(pattern, input_string) 
    # return parts[0]
    out_string = input_string

    # This gets the string up until '\nuser' and discards the rest
    out_string = out_string.split('\nuser')[0]

    # This gets the string up until the first level-1 Markdown header
    out_string = out_string.split('\n#')[0]
    
    # This gets all the text up to the second instance of '\nbot: '
    # It also removes '\n bot: ' from the string
    # pieces = out_string.split('\nbot: ')
    # out_string ='\n'.join(pieces[ 0 : min(len(pieces), 2) ])

    return out_string

display(Markdown(get_substring_until_pattern(result)))

NameError: name 'result' is not defined